### Average Weight as a Baseline Estimator

In [ ]:
import numpy as np

avg_weight = data.weight_pounds.mean()
print("Average Weight: {}".format(round(avg_weight,3)))
rmse = np.sqrt(data.weight_pounds.map(lambda value: (value-avg_weight)**2).mean())
print("RMSE: {}".format(round(rmse,3)))

## Linear Regression with BigQuery

### BQML Step 1: Create BigQuery dataset for ML models

In [ ]:
from google.cloud import bigquery

BQML_DATASET = 'bqml'
BQML_ESTOMATPR_NAME = 'babyweight_estimator'
BQML_DATASET_LOCATION = 'US'

bq_client = bigquery.Client(PROJECT)
dataset_ref = bq_client.dataset(BQML_DATASET)

dataset = bigquery.Dataset(dataset_ref)


if BQML_DATASET in list(map(lambda dataset: dataset.dataset_id,bq_client.list_datasets())):
    print('Deleting BQ Dataset {}...'.format(BQML_DATASET))
    bq_client.delete_dataset(dataset=dataset, delete_contents=True)
    
print('Creating BQ Dataset {}...'.format(BQML_DATASET))
dataset.location = BQML_DATASET_LOCATION
bq_client.create_dataset(dataset=dataset)

print('BQ Dataset {} is up and running'.format(BQML_DATASET))
print("")

### BQML Step 2: Create and train the Linear Regression Model

In [ ]:
from datetime import datetime
import time

bqml_train_query = (
'''
CREATE MODEL {}.{} 
  OPTIONS( model_type='linear_reg',   labels=['weight_pounds'],
    max_iteration=1000,
    learn_rate=0.1, 
    learn_rate_strategy='constant',
    eval_split_method='random',
    eval_split_fraction=0.2,
    early_stop=True,
    l1_reg=0.001
  ) AS
SELECT
  ROUND(weight_pounds,1) AS weight_pounds,
  COALESCE(CAST(is_male AS STRING),'NA') is_male,
  mother_age,
  COALESCE(CAST(mother_race AS STRING),'NA') mother_race,
  plurality,
  gestation_weeks,
  mother_married,
  COALESCE(CAST(cigarette_use AS STRING),'NA') cigarette_use,
  COALESCE(CAST(alcohol_use AS STRING),'NA') alcohol_use
FROM
  publicdata.samples.natality
WHERE
  year = 2000
  AND weight_pounds > 0
  AND mother_age > 0
  AND plurality > 0
  AND gestation_weeks > 0
  AND month > 0
LIMIT
  10000;
'''.format(BQML_DATASET, BQML_ESTIMATOR_NAME)
)

#print bqml_train_query

time_start = datetime.utcnow() 
print("Training started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 

query_job = bq_client.query(
    query=bqml_train_query,
    location=BQML_DATASET_LOCATION
) 
print "Status: {}".format(query_job.state)

try:
    results = query_job.result()
    print results
except:
    pass

print "Status: {}".format(query_job.state)
time_end = datetime.utcnow() 
print(".......................................")
print("Training finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Training elapsed time: {} seconds".format(time_elapsed.total_seconds()))

### BQML Step 3: Get Predictions using the Linear Regression Model

In [ ]:
from datetime import datetime
import time

bqml_predict_query = (
'''
SELECT 
    ROUND(predicted_weight_pounds,1) estimated_weight,
    weight_pounds
FROM ml.PREDICT(
  MODEL {}.{}, 
  (
      SELECT
          ROUND(weight_pounds,1) AS weight_pounds,
          COALESCE(CAST(is_male AS STRING),'NA') is_male,
          mother_age,
          COALESCE(CAST(mother_race AS STRING),'NA') mother_race,
          plurality,
          gestation_weeks,
          mother_married,
          COALESCE(CAST(cigarette_use AS STRING),'NA') cigarette_use,
          COALESCE(CAST(alcohol_use AS STRING),'NA') alcohol_use
      FROM
        publicdata.samples.natality
      WHERE
        year = 2000
        AND weight_pounds > 0
        AND mother_age > 0
        AND plurality > 0
        AND gestation_weeks > 0
          AND month > 0
     LIMIT 10
   )
);

'''.format(BQML_DATASET, BQML_ESTOMATPR_NAME)
)

#print bqml_predict_query

query_job = bq_client.query(
    query=bqml_predict_query,
    location=BQML_DATASET_LOCATION
) 
print "Status: {}".format(query_job.state)

results = query_job.result()
for row in results:
    print("Predicted:{},  Actual: {}".format(row.estimated_weight, row.weight_pounds))

print "Status: {}".format(query_job.state)
